In [18]:
import numpy as np 
import cvxpy as cvx 
import matplotlib.pyplot as plt 
import pdb 

In [2]:
np.random.seed(0)
m=100
k=40 # max # permuted measurements
n=20
A=10*np.random.randn(m,n)
x_true=np.random.randn(n,1) # true x value
y_true = A.dot(x_true) + np.random.randn(m,1)
# build permuted indices
perm_idxs=np.random.permutation(m)
perm_idxs=np.sort(perm_idxs[:k])
temp_perm=np.random.permutation(k)
new_pos=np.zeros(k)
for i in range(k):
  new_pos[i] = perm_idxs[temp_perm[i]]
new_pos = new_pos.astype(int)
# true permutation matrix
P=np.identity(m)
P[perm_idxs,:]=P[new_pos,:]
true_perm=[]
for i in range(k):
  if perm_idxs[i] != new_pos[i]:
    true_perm = np.append(true_perm, perm_idxs[i])
y=P.dot(y_true)
new_pos = None

In [26]:
#Initialize x 
P = np.eye(m)
x, _, _, _ = np.linalg.lstsq(A, P.T.dot(y_true))
res = A.dot(x) - P.T.dot(y_true) 
idx_pre = np.argsort(res.squeeze())[-k:]
# pdb.set_trace()
# P_idx = cvx.Int(k) #indices to which searched indicies will be changed
# P[idx_pre, :] = 0
# P[idx_pre, P_idx] = 1
P_ = cvx.Bool(k,k)
A_ = A[idx_pre, idx_pre]
obj = cvx.Minimize(cvx.norm(A.dot(x) - P_.T*y_true))
constraints = [np.ones(m)*P_*np.ones(m).T <= k,
               P_[idx_pre, :] == P[idx_pre, :]
              ]
prob = cvx.Problem(obj, constraints)
# prob.solve()
# error = []
# for i in range(10):
#     prob.solve()
#     P_ = P.value()
#     x, _, _, _ = np.linalg.lstsq(A, P_.T.dot(y_true))
#     error.append(prob.value())
#     print('Iteration %r solved with error: %r' % (i+1, prob.value()))
    

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])